# MNIST Classifier with HuoguoML

This short notebook shows how to use HuoguoML to deploy and manage a service. This includes:

1. Building and training a neural network to classify MNIST images
2. Create a HuoguoML service

## Requirements

We will be using Tensorflow 2 and HuoguoML. Update `pip` and install packages:

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install huoguoml

Import all packages:

In [1]:
import tensorflow as tf

## Building and training a neural network to classify MNIST images

### Data

First we load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Further, the samples are converted from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Create a neural network
A two layer fully connected neural network, that is trained with cross entropy loss and optimized with adam model:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Train, evaluate and export neural network

Train neural network

In [4]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 787us/step - loss: 0.4780 - accuracy: 0.8610
Epoch 2/5
1875/1875 [==============================] - 1s 737us/step - loss: 0.1498 - accuracy: 0.9556
Epoch 3/5
1875/1875 [==============================] - 1s 715us/step - loss: 0.1118 - accuracy: 0.9653
Epoch 4/5
1875/1875 [==============================] - 1s 702us/step - loss: 0.0860 - accuracy: 0.9749
Epoch 5/5
1875/1875 [==============================] - 1s 701us/step - loss: 0.0730 - accuracy: 0.9776


Evaluate neural network -the image classifier should have a accuracy of ~98% on this dataset.

In [5]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0736 - accuracy: 0.9779


[0.07359115779399872, 0.9779000282287598]

Export neural network

In [6]:
model_dir = "./model"
model.save(model_dir)

INFO:tensorflow:Assets written to: ./model\assets


In [7]:
!saved_model_cli show --dir ./model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT


2021-02-11 14:53:06.924865: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-02-11 14:53:06.924883: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='inputs')
        Argument #2
          DType: bool
          Value: False
        Argument #3
          DType: NoneType
          Value: None
    Option #2
      Callable with:
        Argument #1
          flatten_input: TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='flatten_input')
        Argument #2
          DType: bool
          Value: False
        Argument #3
          DType: NoneType
          Value: None
    Option #3
      Callable with:
        Argument #1
          flatten_input: TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='flatten_input')
        Argument #2
          DType: bool
          Value: True
        Argument #3
          DType: NoneType
          Value: None


## Create a HuoguoML service

In [9]:
import huoguoml

huoguoml.__version__
huoguoml.tensorflow.log_model(tf_saved_model_dir="./model", 
                              tf_meta_graph_tags="serve",
                              tf_signature_def_key="serving_default",
                              artifact_dir="./artifact")

Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...
Validation succeeded!
